In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('dataset/clean_bot.csv')
df

,default_profile,default_profile_image,favourites_count,followers_count,friends_count,geo_enabled,verified,average_tweets_per_day,account_age_days,account_type
0,False,False,4,1589,4,False,False,7.870,1403,bot
1,False,False,536,860,880,False,False,0.183,1379,human
2,False,False,3307,172,594,True,False,0.864,1159,human
3,True,False,8433,517,633,True,False,0.889,1489,human
4,False,False,88,753678,116,True,True,1.339,3138,human
...,...,...,...,...,...,...,...,...,...,...
37433,True,False,651,139,1105,False,False,0.084,4028,human
37434,False,False,8839,1121486,605,True,True,8.976,2782,human
37435,True,False,399,85630,190,False,True,2.226,2773,human
37436,False,False,967,138,166,True,False,0.339,2899,human


In [4]:
df.replace({True: 1, False: 0}, inplace=True)
df.replace({'bot': 1, 'human': 0}, inplace=True)

/tmp/ipykernel_53209/2931568261.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({True: 1, False: 0}, inplace=True)
/tmp/ipykernel_53209/2931568261.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'bot': 1, 'human': 0}, inplace=True)


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
columns_to_standardize = ['favourites_count','followers_count','friends_count','average_tweets_per_day','account_age_days']
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])
df

,default_profile,default_profile_image,favourites_count,followers_count,friends_count,geo_enabled,verified,average_tweets_per_day,account_age_days,account_type
0,0,0,-0.362527,-0.149232,-0.089655,0,0,0.039802,-1.597274,1
1,0,0,-0.346844,-0.149527,-0.071974,0,0,-0.272934,-1.620950,0
2,0,0,-0.265160,-0.149805,-0.077747,1,0,-0.245229,-1.837980,0
3,1,0,-0.114054,-0.149665,-0.076960,1,0,-0.244212,-1.512434,0
4,0,0,-0.360050,0.155160,-0.087395,1,1,-0.225904,0.114308,0
...,...,...,...,...,...,...,...,...,...,...
37433,1,0,-0.343454,-0.149818,-0.067433,0,0,-0.276962,0.992295,0
37434,0,0,-0.102085,0.304022,-0.077525,1,1,0.084799,-0.236887,0
37435,1,0,-0.350883,-0.115218,-0.085901,0,1,-0.189817,-0.245765,0
37436,0,0,-0.334139,-0.149819,-0.086385,1,0,-0.266588,-0.121466,0


In [ ]:
[2,3,4,7,8]

In [6]:
import joblib
joblib.dump(scaler, 'models/scaler.pkl')
scaler = joblib.load('models/scaler.pkl')

In [8]:
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

In [9]:
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8766025641025641
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      5001
           1       0.85      0.77      0.81      2487

    accuracy                           0.88      7488
   macro avg       0.87      0.85      0.86      7488
weighted avg       0.88      0.88      0.88      7488



In [11]:
model.save_model("models/xgboost_model.model")
loaded_model = xgb.Booster()
loaded_model.load_model("models/xgboost_model.model")

/home/stanley/miniconda3/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [23:43:43] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [38]:
test_input =X[0].reshape(-1, 1)
columns_to_standardize = [2,3,4,7,8]
data_to_transform = test_input[columns_to_standardize]
data_to_transform

array([[-0.36252663],
       [-0.14923157],
       [-0.08965519],
       [ 0.03980231],
       [-1.59727362]])

In [36]:
import joblib
import numpy as np
import xgboost as xgb
test_input = X[4].reshape(-1, 1)

loaded_model = xgb.Booster()
loaded_model.load_model("models/xgboost_model.model")

scaler = joblib.load('models/scaler.pkl')
columns_to_standardize = [2,3,4,7,8]
data_to_transform = test_input[columns_to_standardize].reshape(1, -1)
transformed_data = scaler.transform(data_to_transform).reshape(-1, 1)
standardized_input = test_input.copy()
standardized_input[columns_to_standardize] = transformed_data

dtest = xgb.DMatrix(standardized_input)
predictions = loaded_model.predict(dtest)
predictions[0]


/home/stanley/miniconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


0.7987787